In [ ]:
import json
import requests
import tensorflow as tf
import numpy as np 
import cv2
from urllib.request import urlopen
import matplotlib.pyplot as plt

In [ ]:
# Simple parse of the 'games.json' file.

# returns array of a column
def get(collumn):
    url = "https://huggingface.co/datasets/FronkonGames/steam-games-dataset/resolve/main/games.json?download=true"
    response = requests.get(url)
    dataset = json.loads(response.text)

    data = []

    for app in dataset:
        data.append(dataset[app][collumn])

    return data

In [ ]:
img_width = 460
img_height = 215

images = []
est_owners = []

for image, players in zip(get('header_image'), get('estimated_owners')):
    images.append(image)
    est_owners.append(players)
    print(image + ": " + players)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(images)

def get(url):
    with urlopen(str(url.numpy().decode("utf-8"))) as request:
        img_array = np.asarray(bytearray(request.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def read_image_from_url(url):
    return tf.py_function(get, [url], tf.uint8)


dataset_images = dataset.map(lambda x: read_image_from_url(x))

for d in dataset_images:
  print(d)
